大盘指数（基准）

- 深圳成指 399001.SZ

- 上证指数 000001.SH

In [3]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 10)

# 日线数据
df_daily = pd.read_csv("sample_data/daily/000001.SZ.csv.xz")
df_daily.sort_values(by=["trade_date"], inplace=True)
df_daily.reset_index(drop=True, inplace=True)
# 指数日线数据
df_index_daily = pd.read_csv("sample_data/index_daily/399001.SZ.csv.xz")
df_index_daily.sort_values(by=["trade_date"], inplace=True)
df_index_daily.reset_index(drop=True, inplace=True)
# 每日重要的基本面指标
df_daily_basic = pd.read_csv("sample_data/daily_basic/000001.SZ.csv.xz")
df_daily_basic.sort_values(by=["trade_date"], inplace=True)
df_daily_basic.reset_index(drop=True, inplace=True)
# 资产负债表
df_balancesheet = pd.read_csv("sample_data/balancesheet/000001.SZ.csv.xz")
df_balancesheet.sort_values(by=["ann_date"], inplace=True)
df_balancesheet.reset_index(drop=True, inplace=True)
# 财务利润表
df_income = pd.read_csv("sample_data/income/000001.SZ.csv.xz")
df_income.sort_values(by=["ann_date"], inplace=True)
df_income.reset_index(drop=True, inplace=True)
# 财务指标数据
df_fina_indicator = pd.read_csv("sample_data/fina_indicator/000001.SZ.csv.xz")
df_fina_indicator.sort_values(by=["ann_date"], inplace=True)
df_fina_indicator.reset_index(drop=True, inplace=True)

# 交易摩擦类因子

## 市值(firm size， size)

### 【计算方法】

由 **daily_basic** 直接获得【总市值（万元）】

### 【说明】

无

### 【代码示例】

In [72]:
df = pd.DataFrame()

df['trade_date'] = df_daily.trade_date
# 市值
df['total_mv'] = df_daily_basic.total_mv

print('{}: {}'.format(df['trade_date'].iloc[-1], df['total_mv'].iloc[-1]))

20181101: 11847615.8941


## 系统性风险(market beta, beta)

### 【计算方法】

$$
\beta_i=\rho_{i,m} \frac{\sigma_i}{\sigma_m}
$$

$\rho_{i,m}$: 股票 i 的收益率与大盘指数收益率的相关系数

$\sigma_i$: 股票 i 收益率的波动率(即标准差)

$\sigma_m$: 大盘指数收益率的波动率(即标准差)

### 【说明】

#### 基准

大盘指数（基准）

- 深圳成指 399001.SZ

#### 复权

实际计算中需要复权

#### 滚动计算

取过去 12 个月的数据

### 【代码示例】

In [23]:
df = pd.DataFrame()

df['trade_date'] = df_daily.trade_date
df['pct_change'] = df_daily["pct_change"].values
df['pct_change_benchmark'] = df_index_daily["pct_change"]
df.dropna(inplace=True)
cov = np.cov(np.vstack([
            np.array(df["pct_change"].values),
            np.array(df["pct_change_benchmark"].values)
        ]), ddof=1)
print('{}: {}'.format(df['trade_date'].iloc[-1], cov[0][1] / cov[1][1]))

20181101: 0.01703850184053183


## 下行风险 (downside beta，betad)

### 【计算方法】

$$
\beta^- = \frac{Cov(r_i, r_m|r_i < u_m)}{Var(r_m|r_m < u_m)}
$$

$𝑟_i$ 和 $𝑟_m$ 分别代表股票和大盘指数的收益率

$u_m$ 是大盘收益率的均值

### 【说明】

#### 基准

大盘指数（基准）

- 深圳成指 399001.SZ

#### 复权

实际计算中需要复权

#### 滚动计算

取过去 12 个月的数据

### 【代码示例】

In [68]:
df = pd.DataFrame()

df['trade_date'] = df_daily.trade_date
df['pct_change'] = df_daily["pct_change"]
df['pct_change_benchmark'] = df_index_daily["pct_change"]
df.dropna(inplace=True)

# u_m
ave_pct_change_benchmark = np.mean(df['pct_change_benchmark'])

# calculate var
var = np.var(df[df['pct_change_benchmark'] < ave_pct_change_benchmark]['pct_change_benchmark'], ddof=1)
print('[var]: {}'.format(var))

# calculate cov
df['pct_change'] = df[df['pct_change'] < ave_pct_change_benchmark]['pct_change']
df.dropna(inplace=True)
cov = np.cov(np.vstack([
            np.array(df["pct_change"].values),
            np.array(df["pct_change_benchmark"].values)
        ]), ddof=1)
print('[cov]: {}'.format(cov[0][1]))

print('[{}]: {}'.format(df['trade_date'].iloc[-1], cov[0][1] / var))

[var]: 2.2210715724726264
[cov]: 0.038248650969275565
[20181101]: 0.017220809740361016


## 特定波动率 (idiosyncratic volatility, idvol)

### 【计算方法】

将股票收益率对市场大盘指数收益率进行回归，所得残差的标准差即为特定波动率

$$
r_{i,t} = \alpha_i + \beta_i * r_{m,t} + \epsilon_{i,t}
$$

$𝑟_i$ 和 $𝑟_m$ 分别代表股票和大盘指数的收益率

### 【说明】

#### 基准

大盘指数（基准）

- 深圳成指 399001.SZ

#### 复权

实际计算中需要复权

#### 滚动计算

取过去 12 个月的数据

### 【代码示例】

In [70]:
import statsmodels.formula.api as sm

df = pd.DataFrame()

df['trade_date'] = df_daily.trade_date
df['pct_change'] = df_daily["pct_change"]
df['pct_change_benchmark'] = df_index_daily["pct_change"]
df.dropna(inplace=True)

model = sm.ols(formula = 'pct_change ~ pct_change_benchmark',data = df).fit()
resid = model.resid
print('[resid]:\n{}'.format(resid))

print('[{}]: {}'.format(df['trade_date'].iloc[-1], np.sqrt(np.var(resid))))

[resid]:
1      -0.564846
2      -0.564977
3      -1.064877
4      -0.574851
5      -0.574790
          ...   
6522   -1.040566
6523   -3.939371
6524    1.301618
6525   -0.012900
6526   -0.832251
Length: 6526, dtype: float64
[20181101]: 2.74315920527903


## 总波动率(total volatility, vol)

### 【计算方法】

股票日收益率的标准差

### 【说明】

#### 复权

实际计算中需要复权

#### 滚动计算

取过去 12 个月的数据

### 【代码示例】

In [90]:
df = pd.DataFrame()

df['trade_date'] = df_daily.trade_date
df['pct_change'] = df_daily["pct_change"]
df.dropna(inplace=True)

print('[{}]: {}'.format(df['trade_date'].iloc[-1], np.std(df['pct_change'])))

[20181101]: 2.7547858711294775


## 特定偏态 (idiosyncratic skewness, idskew)

### 【计算方法】

特定偏态的计算过程与与**特定波动率**基本一致，区别在于计算残差的偏态而非标准差。

### 【说明】

#### 基准

大盘指数（基准）

- 深圳成指 399001.SZ

#### 复权

实际计算中需要复权

#### 滚动计算

取过去 12 个月的数据

### 【代码示例】

In [88]:
import statsmodels.formula.api as sm
from scipy.stats import skew

df = pd.DataFrame()

df['trade_date'] = df_daily.trade_date
df['pct_change'] = df_daily["pct_change"]
df['pct_change_benchmark'] = df_index_daily["pct_change"]
df.dropna(inplace=True)

model = sm.ols(formula = 'pct_change ~ pct_change_benchmark',data = df).fit()
resid = model.resid
print('[resid]:\n{}'.format(resid))
print('[{}]: {}'.format(df['trade_date'].iloc[-1], skew(resid)))

[resid]:
1      -0.564846
2      -0.564977
3      -1.064877
4      -0.574851
5      -0.574790
          ...   
6522   -1.040566
6523   -3.939371
6524    1.301618
6525   -0.012900
6526   -0.832251
Length: 6526, dtype: float64
[20181101]: 0.9750345704069766


## 总偏态 (total skewness, skew12)

### 【计算方法】

股票日收益率的偏态。

### 【说明】

#### 复权

实际计算中需要复权

#### 滚动计算

取过去 12 个月的数据

### 【代码示例】

In [91]:
from scipy.stats import skew

df = pd.DataFrame()

df['trade_date'] = df_daily.trade_date
df['pct_change'] = df_daily["pct_change"]
df.dropna(inplace=True)

print('[{}]: {}'.format(df['trade_date'].iloc[-1], skew(df['pct_change'])))

[20181101]: 0.9051220455030352


## 共同偏态 (coskewness, coskew12)

### 【计算方法】

选用第二种。

[Harvey and Siddique (2000)]

$$
coskew = \frac{E[\epsilon_i\epsilon_m^2]}{\sqrt{E[\epsilon_i^2]}E[\epsilon_m^2]}
$$

[Ang, Chen, and Xing (2006)]

$$
coskew = \frac{E[(r_i-\mu_i)(r_m-\mu_m)^2]}{\sqrt{var(r_i)}var(r_m)}
$$

$𝑟_i$ 和 $𝑟_m$ 分别代表股票和大盘指数的收益率

### 【说明】

#### 基准

大盘指数（基准）

- 深圳成指 399001.SZ

#### 复权

实际计算中需要复权

#### 滚动计算

取过去 12 个月的数据

### 【代码示例】

In [93]:
from scipy.stats import skew

df = pd.DataFrame()

df['trade_date'] = df_daily.trade_date
df['pct_change'] = df_daily["pct_change"]
df['pct_change_benchmark'] = df_index_daily["pct_change"]
df.dropna(inplace=True)

ave_pct_change = np.mean(df['pct_change'])
ave_pct_change_benchmark = np.mean(df['pct_change_benchmark'])
numerator = np.mean((df['pct_change'] - ave_pct_change) * np.square(df['pct_change_benchmark'] - ave_pct_change_benchmark))
denominator = np.sqrt(np.var(df['pct_change']))*np.var(df['pct_change_benchmark'])

print('[{}]: {}'.format(df['trade_date'].iloc[-1], numerator / denominator))

[20181101]: 0.11987603507199311


# 动量因子

## 12 个月动量 (12-month momentum, mom12)

### 【TODO】

【未完成原因】balabala